important questions

- how do i use image segmentation in conjunction with classification?

- is it any better to train multiple binary classifiers instead of a multiclass CNN

- apparently i can only use data augmentation if i do end-to-end? i guess i cant store too many augmented images in system memory but i can probably find a way around it

TODO:

- class weighting
- fine tuning
- xgboost
- image cleaning: 
    - removing background? prob not gonna work
    - lighting adjustment - read other papers
    
- single classifiers
- progressive resizing
- hyperparam search? (at the very end)

DONE:
- batch normalization <- only really for speedups
    
Findings:

- data augmentation on underrepresented classes is better than just throwing everything at the model i guess thats obvious

In [1]:
import numpy as np
import os
import cv2
import sys

def loading_text(text):
    sys.stdout.write(str(text) + '\r')
    sys.stdout.flush()

def load_img(img_name):
    img = cv2.imread(os.path.join(img_dir, img_name), 1)
    img = cv2.resize(img, dsize=(300,225))
    img = img[1:, 38:-38]
#     img = cv2.resize(img, dsize=(200,150))
    return img

def extract_features(X, batch_size, conv_base):
    total = len(X)
    output_shape = conv_base.layers[-1].output_shape[1:]
    features = np.zeros(shape=(total,) + output_shape)
    i = 0
    while i*batch_size < total:
        loading_text(str(i*batch_size) +  "/" + str(total))
        inputs_batch = X[i*batch_size:(i+1)*batch_size]/255 # SCALING TO 0-1 HERE
        features[i * batch_size : (i + 1) * batch_size] = conv_base.predict(inputs_batch)
        i += 1
    return features.reshape(total, np.prod(output_shape))

In [2]:
img_dir = "data\\ISIC2018_Task3_Validation_Input"
img_names = np.array(os.listdir(img_dir))
X_all = np.array([load_img(img_name) for img_name in img_names])

In [3]:
# from keras.applications import VGG16
# conv_base = VGG16(weights='imagenet',
#                   include_top=False,
#                   input_shape=(150, 200, 3))
from keras.applications.mobilenet_v2 import MobileNetV2
conv_base = MobileNetV2(include_top=False, weights='imagenet', input_shape=(224, 224, 3))

#conv_base.summary()

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.


In [4]:
X_f = extract_features(X_all, 32, conv_base)

In [5]:
X_f.shape

(193, 62720)

In [6]:
from keras.models import load_model
from keras import backend as K

def recall_threshold(threshold = 0.5):
    def recall(y_true, y_pred):
        """Recall metric.
        Computes the recall over the whole batch using threshold_value.
        """
        threshold_value = threshold
        # Adaptation of the "round()" used before to get the predictions. Clipping to make sure that the predicted raw values are between 0 and 1.
        y_pred = K.cast(K.greater(K.clip(y_pred, 0, 1), threshold_value), K.floatx())
        # Compute the number of true positives. Rounding in prevention to make sure we have an integer.
        true_positives = K.round(K.sum(K.clip(y_true * y_pred, 0, 1)))
        # Compute the number of positive targets.
        possible_positives = K.sum(K.clip(y_true, 0, 1))
        recall_ratio = true_positives / (possible_positives + K.epsilon())
        return recall_ratio
    return recall

In [13]:
model = load_model("models\\0003.h5")#, custom_objects={'recall':recall_threshold()})
# model.summary()

In [14]:
a = model.predict(X_f)
b = np.zeros_like(a)
b[np.arange(len(a)), a.argmax(1)] = 1

In [15]:
import csv
def write_csv(csv_name, img_names, Y_pred):
    with open(csv_name, 'w') as csvfile:
        filewriter = csv.writer(csvfile, delimiter=',', lineterminator='\n',
                                quotechar='|', quoting=csv.QUOTE_MINIMAL)
        filewriter.writerow(['image', 'MEL', 'NV', 'BCC', 'AKIEC', 'BKL', 'DF', 'VASC'])
        for i in range(len(img_names)):
            filewriter.writerow([img_names[i][:-4]] + Y_pred[i].tolist())

In [16]:
write_csv('ISIC_2018_Validation.csv', img_names, b)